#Deep Learning - Project

*Irland news headlines data*

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import regex
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
import plotly.express as px
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer

# https://www.kaggle.com/code/darkrubiks/classifying-irish-times-headlines-categories#Predicting-the-Headlines-Categories

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# !ls "/content/drive/MyDrive/Untitled folder/ireland-news-headlines.csv"

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
data = pd.read_csv('/content/drive/MyDrive/Untitled folder/ireland-news-headlines.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Untitled folder/ireland-news-headlines.csv'

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d therohk/ireland-historical-news


In [ ]:
from zipfile import ZipFile
with ZipFile('/content/ireland-historical-news.zip','r') as zip0bj:
  zip0bj.extractall('/content')

In [ ]:
data = pd.read_csv('/content/ireland-news-headlines.csv')
data

### Importing the data

## Data Exploration

In [ ]:
data.shape


In [ ]:
data.isna().sum()

#We can see that we have 7 nulls in the headline text, we will remove them

In [ ]:
#removing the missing text articles
data = data.dropna()
data.isna().sum()

In [ ]:
data.duplicated().sum()
#Check what to do with it###

In [ ]:
#Checking how many istanses we have of each category over the years in the data set
data["headline_category"].value_counts()

### Data Pre-Processing

In [ ]:
#Date modifications
#We will change the publish date to date time format
#We will add month and day date for further analyzing
data.loc[:, 'publish_date'] = pd.to_datetime(data['publish_date'], format='%Y%m%d')
data.loc[:, 'month'] = data['publish_date'].dt.month
data.loc[:, 'day'] = data['publish_date'].dt.day
data

In [ ]:
# Unifing sub-categories into main categories by first delimter of "."
def split_point(x):
  split = x.split(".")
  x = split[0]
  return x

data["headline_category"] = data["headline_category"].apply(split_point)
data["headline_category"].value_counts()
#we are left with 6 main categories news : sport, business, opinion ,culture, lifestyle

# We Can see that there are almost 800K rows of news which cause over fittting and imbalanced data.
# So we decided to balance our data by deleting X rows from News category and adding Y rows to culture, opinion, lifestyle...

In [ ]:
news_delete = data[data["headline_category"]=="news"].sample(n=702680).index
data = data.drop(news_delete)
data["headline_category"].value_counts()

In [ ]:
news_delete = data[data["headline_category"]=="sport"].sample(n=165826).index
data = data.drop(news_delete)
data["headline_category"].value_counts()


In [ ]:
news_delete = data[data["headline_category"]=="business"].sample(n=126960).index
data = data.drop(news_delete)
data["headline_category"].value_counts()


In [ ]:
news_delete = data[data["headline_category"]=="culture"].sample(n=2944).index
data = data.drop(news_delete)
data["headline_category"].value_counts()


In [ ]:
news_delete = data[data["headline_category"]=="opinion"].sample(n=37073).index
data = data.drop(news_delete)
data["headline_category"].value_counts()

In [ ]:
data

In [ ]:
# Plotting a bar chart to see the overall appearancses of each category after the unfigying we made
counts = data['headline_category'].value_counts()

# color map for the bars
colors = ['blue', 'green', 'red', 'purple', 'orange','magenta']

# plot the result using a bar chart with custom colors
counts.plot.bar(color=colors[:len(counts)])

# set plot title and axis labels
plt.title('Number of Appearances by Category')
plt.xlabel('Category')
plt.ylabel('Number of appearances')
plt.xticks(rotation=0)

# show the plot
plt.show()

In [ ]:
#Plotting a trend graph over the years of publish dates to see categories volume trend

grouped_data = data.groupby([data['publish_date'].dt.year, 'headline_category']).size().unstack()

# Plot the data
grouped_data.plot(kind='line', figsize=(15, 5))
plt.xlabel('Publish Year')
plt.ylabel('Number of Articles')
plt.title('Trend of Categories Over The Years')
plt.legend(title='Category', loc = "upper right")
plt.show()

#We can see that news articles are the most common over the years by far. also can see a big decrease in the volumne of the articles from 2010 in all categories


In [ ]:
#Plotting a trend graph over the years of publish dates to see categories volume trend
data_avg = data
data_avg["num"] = 1
data.set_index("month")
grouped_data = data_avg.groupby(["month", "headline_category"])["num"].mean()
grouped_data

In [ ]:
#Plotting a trend graph over the years of publish dates to see categories volume trend

grouped_data = data.groupby([data['day'], 'headline_category']).size().unstack()

# Plot the data
grouped_data.plot(kind='line', figsize=(15, 5))
plt.xlabel('Publish Day')
plt.ylabel('Number of Articles')
plt.title('Trend of Categories Over The Days')
plt.legend(title='Category', loc = "upper right")
plt.show()

#We can see that news articles are the most common over the years by far. also can see a big decrease in the volumne of the articles from 2010 in all categories


In [ ]:
#Plotting a trend graph over the years of publish dates to see categories volume trend

grouped_data = data.groupby([data['month'], 'headline_category']).size().unstack()

# Plot the data
grouped_data.plot(kind='line', figsize=(15, 5))
plt.xlabel('Publish Month')
plt.ylabel('Number of Articles')
plt.title('Trend of Categories Over The Months')
plt.legend(title='Category', loc = "upper right")
plt.show()

#We can see that news articles are the most common over the years by far. also can see a big decrease in the volumne of the articles from 2010 in all categories

In [ ]:
# Removing stop words using the WordNetLemmatizer library

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

wordnet = WordNetLemmatizer()
stop_words = stopwords.words('english')

for punct in punctuation:
    stop_words.append(punct)

def filter_text(text, stop_words):
    if isinstance(text, str):
        word_tokens = WordPunctTokenizer().tokenize(text.lower())
        filtered_text = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha()]
        filtered_text = [wordnet.lemmatize(w, pos="v") for w in filtered_text if not w in stop_words]
        return " ".join(filtered_text)
    else:
        return ""
data['headline_text'] = data['headline_text'].apply(lambda x: filter_text(x, stop_words))


In [ ]:
def words_frequency(x):
  data_category = data[data["headline_category"] == x]
  data_words_category = data_category["headline_text"].str.split(expand=True).stack().value_counts()
  data_words_category = pd.DataFrame(data_words_category)
  data_words_category = data_words_category.rename(columns={ 0: "count"})
  #data_words_category = data_words_category(columns={ 0: "count"}, errors="raise")
  data_words_category = data_words_category.reset_index()
  data_words_category = data_words_category.rename(columns={ "index": "words"}, errors="raise")
  headline = x.capitalize() + ' Words Frequency'
  fig = px.treemap(data_words_category[0:100], path=[px.Constant(headline), 'words'],
                     values='count',
                     color='count',
                     color_continuous_scale='viridis',
                     color_continuous_midpoint=np.average(data_words_category['count'])
                     )
  fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
  fig.show()

x='news'
#headline = x.capitalize() + ' Words Frequency'
words_frequency(x)


'''
data_news = data[data["headline_category"] == "news"]
data_words_news = data_news["headline_text"].str.split(expand=True).stack().value_counts()
data_words_news = pd.DataFrame(data_words_news)
data_words_category = data_words_category.rename(columns={ 0: "count"})
#data_words_news = data_words_news.rename(columns={ 0: "count"}, errors="raise")
data_words_news = data_words_news.reset_index()
data_words_news = data_words_news.rename(columns={ "index": "words"}, errors="raise")
fig = px.treemap(data_words_news[0:100], path=[px.Constant("News Words Frequency"), 'words'],
                 values='count',
                 color='count',
                 color_continuous_scale='viridis',
                 color_continuous_midpoint=np.average(data_words_news['count'])
                )
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()
'''




In [ ]:
words_frequency('business')

In [ ]:
words_frequency('sport')

In [ ]:
words_frequency('culture')

In [ ]:
words_frequency('opinion')

In [ ]:
words_frequency('lifestyle')

In [ ]:
# Word To Vector
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

# Load the data

# Split the headline text into words and store them as a list
sentences = [headline.split() for headline in data["headline_text"].tolist()]

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

from collections import Counter

word_counts = Counter([word for sentence in sentences for word in sentence])
#min_count = 7500 # Change this value to set the minimum number of occurrences for a word to be included in the model
#min_count1 = 9000
min_count1 = 1000
#max_count = 10000 # Change this value to set the maximum number of occurrences for a word to be included in the model
#max_count1 = 20000
max_count1 = 30000
#filtered_sentences = [[word for word in sentence if min_count <= word_counts[word] <= max_count] for sentence in sentences]
filtered_sentences1 = [[word for word in sentence if min_count1 <= word_counts[word] <= max_count1] for sentence in sentences]
# Train Word2Vec model
from gensim.models import Word2Vec

#model = Word2Vec(filtered_sentences, vector_size=100, window=3, min_count=min_count, workers=4)
model1 = Word2Vec(filtered_sentences1, vector_size=100, window=3, min_count=min_count1, workers=4)

X = model1.wv[model1.wv.key_to_index]
pca = PCA(n_components=2)
result = pca.fit_transform(X)

# Plot the 2D word vector space
fig, ax = pyplot.subplots()
ax.scatter(result[:, 0], result[:, 1])
words = list(model1.wv.key_to_index.keys())
for i, word in enumerate(words):
    ax.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

The Word2Vec model represents words as high-dimensional vectors, typically with several hundred dimensions. This means that each word is represented by a point in a space that has several hundred axes, with each axis representing a different aspect of the word's meaning.

However, it is difficult for us to visualize points in a space with more than three dimensions. Therefore, in order to make the word vectors more easily interpretable, we need to reduce the dimensionality of the space to two or three dimensions.

This is where PCA comes in. PCA is a mathematical technique that takes the high-dimensional word vectors and finds the two directions in the space that explain the most variation in the data. These two directions are called the principal components, and they are chosen in such a way that the first principal component captures the most variation in the data, and the second principal component captures the most variation that is orthogonal (perpendicular) to the first principal component.

So, when we plot the word vectors in the two-dimensional space defined by the two principal components, each word is represented by a point in this 2D space, with its x-coordinate corresponding to its projection onto the first principal component, and its y-coordinate corresponding to its projection onto the second principal component.

**By visualizing the word vectors in this 2D space, we can gain insights into the relationships between words based on their similarities and differences in meaning. For example, words that are semantically similar will be closer to each other in the 2D space, while words that are semantically dissimilar will be further apart.**

In [ ]:
# tokenize headlines into sentences
sentences = []
for headline in data['headline_text']:
    sentences.extend(nltk.sent_tokenize(headline))

# get lengths of each sentence
sentence_lengths = [len(nltk.word_tokenize(sentence)) for sentence in sentences]

# calculate summary statistics
mean_length = sum(sentence_lengths) / len(sentence_lengths)
std_dev = np.std(sentence_lengths)

# create histogram of sentence lengths
plt.hist(sentence_lengths, bins=10)
plt.title("Distribution of Sentence Lengths in Headlines")
plt.xlabel("Sentence Length (in words)")
plt.ylabel("Frequency")
plt.show()

# print summary statistics
print("Mean sentence length:", mean_length)
print("Standard deviation of sentence lengths:", std_dev)

In [ ]:
#################
tokenized_sentences = [nltk.word_tokenize(sentence.lower()) for sentence in sentences] #BE AWARE THAT I ALREADY SPLIT THE DATA
 #!!!#                                                                                      #TO A LIST OF WORDS SO THERE IS A CHANCE
# Extract the vocabulary and embeddings from the Word2Vec model                        #THAT I TOKENIZED TWICE
vocabulary = model1.wv.index_to_key
embeddings = model1.wv.vectors
co_occurrence_matrix = np.zeros((len(vocabulary), len(vocabulary)))

# Set the window size for co-occurrence
window_size = 2

#Build the co-occurence matrix
for sentence in tokenized_sentences:
    for i, word in enumerate(sentence):
        if word in vocabulary:
            context = sentence[max(0, i - window_size) : i] + sentence[i + 1 : i + window_size + 1]
            for context_word in context:
                if context_word in vocabulary:
                    co_occurrence_matrix[model1.wv.key_to_index[word]][model1.wv.key_to_index[context_word]] += 1

# Normalize the co-occurrence matrix
co_occurrence_matrix = co_occurrence_matrix / np.diag(co_occurrence_matrix)

In [ ]:
#######################
from sklearn.model_selection import train_test_split

# sentences is a list of tokenized sentences
train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)

# train the Word2Vec model on the training data
model = Word2Vec(train_sentences, vector_size=100, window=3, min_count=5, workers=4)

# evaluate the model on the testing data
accuracy = model.evaluate(test_sentences)

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers datasets

TEST FOR Bert Training



In [ ]:
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import torch
import torch.nn.utils.prune as prune
from datasets import load_dataset, load_metric
from torch.utils.data import TensorDataset, DataLoader, Dataset
from sklearn import preprocessing

In [ ]:
#was df = data
df= data
model_name = 'albert-base-v2'
#model_name = 'bert-base-uncased'
num_of_labels = len(data['headline_category'].unique())
model_seq_classification = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_of_labels)
optimizer = torch.optim.Adam(model_seq_classification.parameters(), lr=1e-5)
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
for param in list(model_seq_classification.named_parameters()):
    print(param)
    prune.l1_unstructured(model_seq_classification.albert.embeddings.word_embeddings, name="weight", amount=3)
for buffer in model_seq_classification.named_buffers():
    print(buffer)

In [ ]:
class rumor_ds(Dataset):
    def __init__(self, data, transform=None):
        self.df = data
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        x = self.df.iloc[index]['headline_text']
        label = self.df.iloc[index]['headline_category']

        if self.transform:
            for trans in self.transform:
                #x, attention_mask = trans(x)
                x = trans(x)
        return x, label


class Tokenize():
    def __init__(self, model_name, max_length=128):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def __call__(self, sample):
        data = self.tokenizer(sample, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        return data
        #return torch.tensor(data['input_ids']), torch.tensor(data['attention_mask'])

In [ ]:
# Cutting the data to 5% so we can run it
# Define the percentage of data to be randomly selected
# Define the percentage of data to be randomly selected
percentage = 0.05

# Group the data by category
grouped_df = df.groupby('headline_category')

# Create an empty dataframe to store the randomly selected data
new_df = pd.DataFrame()

# Iterate over each category
for category, group in grouped_df:
    # Calculate the number of rows to be selected for this category
    num_rows = int(len(group) * percentage)

    # Randomly select the rows
    random_rows = group.sample(n=num_rows, random_state=42)  # Setting random_state for reproducibility

    # Append the randomly selected rows to the new dataframe
    new_df = new_df.append(random_rows)

# Reset the index of the new dataframe
new_df.reset_index(drop=True, inplace=True)

# Display the new dataframe
new_df["headline_category"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and validation sets
train_data, val_data = train_test_split(new_df, test_size=0.2, random_state=42)

In [ ]:
torch.cuda.empty_cache()
train_dataset = rumor_ds(train_data, [Tokenize(model_name)])
#train_dataset = rumor_ds(train_data, [Tokenize(model_name)])
val_dataset = rumor_ds(val_data, [Tokenize(model_name)])
#val_dataset = rumor_ds(train_data, [Tokenize(model_name)])
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_seq_classification.to(device)

In [ ]:
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

count = 0
le = preprocessing.LabelEncoder()
labels = df['headline_category'].unique()
le.fit(labels)

train_accuracies = []
test_accuracies = []

train_losses = []
test_losses = []

for e in range(epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    start_time = time.time()

    # Training phase
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {e + 1}", leave=False)

    for inputs, targets in progress_bar:
        targets = torch.LongTensor(le.transform(targets))
        inputs = {key: value.squeeze(1).to(device) for key, value in inputs.items()}
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model_seq_classification(**inputs, labels=targets)
        batch_loss = outputs.loss
        batch_loss.backward()
        optimizer.step()

        total_loss += batch_loss.item()
        total_correct += (outputs.logits.argmax(dim=1) == targets).sum().item()
        total_samples += targets.size(0)

        progress_bar.set_postfix(loss=f"{total_loss / total_samples:.4f}", accuracy=f"{total_correct / total_samples:.4f}")

    train_loss = total_loss / len(train_dataloader)
    train_accuracy = total_correct / total_samples

    # Validation phase
    model_seq_classification.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_inputs, val_targets in val_dataloader:
            val_targets = torch.LongTensor(le.transform(val_targets))
            val_inputs = {key: value.squeeze(1).to(device) for key, value in val_inputs.items()}
            val_targets = val_targets.to(device)

            val_outputs = model_seq_classification(**val_inputs, labels=val_targets)
            val_batch_loss = val_outputs.loss

            val_loss += val_batch_loss.item()
            val_correct += (val_outputs.logits.argmax(dim=1) == val_targets).sum().item()
            val_total += val_targets.size(0)

    val_loss /= len(val_dataloader)
    val_accuracy = val_correct / val_total

    elapsed_time = time.time() - start_time

    print(f"Epoch {e+1}:")
    print(f"Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")
    print(f"Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")
    print(f"Elapsed time: {elapsed_time:.2f} seconds")

    train_accuracies.append(train_accuracy)
    test_accuracies.append(val_accuracy)

    train_losses.append(train_loss)
    test_losses.append(val_loss)

# Create plots
plt.figure(figsize=(10, 5))

# Train Accuracy VS Test Accuracy
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, epochs + 1), test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train Accuracy VS Test Accuracy')
plt.legend()

# Train Loss VS Test Loss
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, epochs + 1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train Loss VS Test Loss')
plt.legend()

# Adjust the layout to prevent overlapping of subplots
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
headline = 'Your headline goes here'
headline_encoding = tokenizer.encode_plus(headline, truncation=True, padding=True, return_tensors='tf')
input_ids = headline_encoding['input_ids']
attention_mask = headline_encoding['attention_mask']

input_ids = np.array(input_ids)
attention_mask = np.array(attention_mask)

predictions = model.predict([input_ids, attention_mask])
predicted_label_index = np.argmax(predictions, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_label_index])[0]

print('Predicted label:', predicted_label)